In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from scipy.optimize import minimize
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the CSV file from your Google Drive
file_path = '/content/drive/MyDrive/Unwarped_phase/Newtry/NewYY.csv'
data = pd.read_csv(file_path)

# Extract the relevant columns
latitudes = data['Lat'].values
longitudes = data['Long'].values
interferogram_distances = data['Interferogram'].values

# Reshape the data for GMM
interferogram_distances_reshaped = interferogram_distances.reshape(-1, 1)

# Fit the GMM
K = 3  # Number of Gaussian components, adjust as needed
gmm = GaussianMixture(n_components=K)
gmm.fit(interferogram_distances_reshaped)

# Dummy data for B, a, and H
# Adjust B to match the length of the dataset
B = np.random.rand(len(interferogram_distances), 3)  # Adjust dimensions as needed

# L: Observation vector (LOS displacements)
L = interferogram_distances  # Ensure L matches the length of the dataset

# a: Smoothing factor
a = 0.1

# H: Smoothing matrix (second-order finite-difference operator)
H = np.array([
    [1, -2, 1],
    [-2, 4, -2],
    [1, -2, 1]
])

# Define the Okada model (dummy implementation)
def okada_model(S, latitudes, longitudes):
    # Compute displacements using the Okada model
    # This is a dummy implementation, replace with actual model
    displacements_east = np.full(latitudes.shape, S[0])
    displacements_north = np.full(latitudes.shape, S[1])
    displacements_up = np.full(latitudes.shape, S[2])
    return displacements_east, displacements_north, displacements_up

# Define the objective function
def objective_function(S, B, L, a, H, gmm, latitudes, longitudes):
    # Compute the predicted displacements using the Okada model
    predicted_displacements_east, predicted_displacements_north, predicted_displacements_up = okada_model(S, latitudes, longitudes)

    # Concatenate the predicted displacements into a single array
    predicted_displacements = np.column_stack((predicted_displacements_east, predicted_displacements_north, predicted_displacements_up))

    # Ensure B has the correct dimensions
    B_extended = np.tile(B, (len(latitudes), 1))

    # Compute the data misfit term (least squares)
    data_misfit = np.linalg.norm(B_extended @ predicted_displacements.T - L)**2

    # Compute the smoothing term (regularization)
    smoothing_term = a**2 * np.linalg.norm(H @ S)**2

    # Compute the GMM log-likelihood term
    log_likelihood = np.sum(gmm.score_samples(S.reshape(-1, 1)))

    # Combine the terms into the objective function to minimize
    return data_misfit + smoothing_term - log_likelihood

# Initial guess for S
initial_guess = np.zeros(3)  # Adjust length as needed

# Perform optimization using the L-BFGS-B optimizer
result = minimize(objective_function, initial_guess, args=(B, L, a, H, gmm, latitudes, longitudes), method='L-BFGS-B')
optimal_slip_distribution = result.x

print("Optimal Slip Distribution:", optimal_slip_distribution)

# Compute 3D displacements using the Okada model
displacements_east, displacements_north, displacements_up = okada_model(optimal_slip_distribution, latitudes, longitudes)

# Placeholder values for incidence and heading angles (adjust as needed)
incidence_angle = 34  # Example value in degrees
heading_angle = -168  # Example value in degrees

# Define the LOS displacement function
def los_displacement(east_displacement, north_displacement, up_displacement, incidence_angle, heading_angle):
    # Convert angles to radians
    incidence_angle_rad = np.radians(incidence_angle)
    heading_angle_rad = np.radians(heading_angle)

    # Compute the LOS displacement
    los_displacement = (-np.sin(incidence_angle_rad) * np.cos(heading_angle_rad) * east_displacement
                        - np.sin(incidence_angle_rad) * np.sin(heading_angle_rad) * north_displacement
                        + np.cos(incidence_angle_rad) * up_displacement)
    return los_displacement

# Compute the LOS displacement
los_displacements = los_displacement(displacements_east, displacements_north, displacements_up, incidence_angle, heading_angle)

# Combine the 3D displacements into a single array
combined_displacements = np.vstack((displacements_east, displacements_north, displacements_up, los_displacements)).T

# Create a DataFrame for easier manipulation and visualization
df_displacements = pd.DataFrame({
    'Lat': latitudes,
    'Long': longitudes,
    'East_Displacement': displacements_east,
    'North_Displacement': displacements_north,
    'Up_Displacement': displacements_up,
    'LOS_Displacement': los_displacements
})

# Save the DataFrame to a CSV file
output_file_path = '/content/drive/MyDrive/Unwarped_phase/Newtry/3D_Displacements.csv'
df_displacements.to_csv(output_file_path, index=False)

print(f"3D displacements saved to {output_file_path}")


Mounted at /content/drive
Optimal Slip Distribution: [-0.01806503 -0.0206323  -0.01860666]
3D displacements saved to /content/drive/MyDrive/Unwarped_phase/Newtry/3D_Displacements.csv
